In [15]:
"""
This is an implementation of a baseline for low resource language intent classier model.
The model recognizes a sequence of phonemes and output an intent. 

To emulate a low resource situation, the model is trained and evaluated on different combinations of speakers and  utterances.
Starting with one speaker and one utterance up to 7 speakers and 7 utterances. Basically the model is trained and evaluated on 49
different combinations.

The notebook runs a grid search of combinations of speakers and utterrances 
may take a while. Sometimes up to 4 hours
"""

'\nThis notebook is for baseline. It can be run from top to bottom.\nIt runs a grid search of combinations of speakers and utterrances \nmay take a while. Sometimes up to 4 hours\n'

In [16]:
!pip install gdown

In [17]:
!gdown --fuzzy https://drive.google.com/file/d/1LIrogRWSL-4CifdzciM6vV8V30JArQG6/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1LIrogRWSL-4CifdzciM6vV8V30JArQG6
To: /content/phonemes.zip
100% 5.22M/5.22M [00:00<00:00, 328MB/s]


In [18]:
!unzip -qn ./phonemes.zip
!rm -rf '/kaggle/working/phonemes/validation/pp10/spchdatadir/recording1/Untitled.ipynb'
!rm -rf '/content/phonemes/validation/pp10/spchdatadir/recording1/Untitled.ipynb'

In [19]:
!pip install torchsummaryX 

In [20]:
import numpy as np 
import pandas as pd 
import os
import os.path as osp
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummaryX import summary
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from tqdm import tqdm
import random

torch.manual_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [21]:
#We use 2 and 4 intents classification for our final report
intents_6 = ["move", 'turn', 'approach', 'grab', 'point', 'lift']
intents_4 = ['approach', 'grab', 'point', 'lift']
intents_2 = ['approach', 'lift']
intents_3 = ['approach', 'grab', 'point']


In [22]:
kaggle_path = '/kaggle/working/phonemes/'
colab_path = '/content/phonemes/'

X_dir = colab_path + 'train/*/spchdatadir/*/*'
X_dir_val = colab_path + 'validation/*/spchdatadir/*/*'
X_dir_test = colab_path + 'test/*/spchdatadir/*/*'

X_files_train = sorted(glob.glob(X_dir))
X_files_val = sorted(glob.glob(X_dir))
X_files_test = sorted(glob.glob(X_dir))

files = X_files_train
files.extend(X_files_val)
files.extend(X_files_test)

phones = set()

for x in files:
    f = np.load(x)
    phones.update(f)
    
PHONEMES = list(phones)

In [23]:
#A utility function to get number of required number of records
def records(lis, num):
    k = []
    for i in range(0, len(lis), 15):
        k.extend(lis[i:i+num])
    return k

#A utility function to pull files depending on the list of intents chosen
def intents_func(intents_lis, X_files, Y_files):
    xfiles = []
    yfiles = []
    for i, _file in enumerate(Y_files):         
        f = open(_file) 
        intent = f.read()   
        if intent in intents_lis:
            xfiles.append(X_files[i])
            yfiles.append(_file)
    return xfiles, yfiles, intents_lis

#A utility function to retrieve speakers required
def choose_speakers(speakers_lis, xlis, ylis, n=7):
    speakers = random.choices(speakers_lis, k=n)
    x_train_files = []
    y_train_files = []
    for i, _file in enumerate(xlis):
        dirs = _file.split('/')
        for speaker in speakers:
            if speaker in dirs:
                x_train_files.append(_file)
                y_train_files.append(ylis[i]) 
    return x_train_files, y_train_files

In [24]:
speakers = ['pp2', 'pp3', 'pp4', 'pp5', 'pp6', 'pp7', 'pp8']

In [25]:
partition_train= "train"
partition_validate= "validation"

X_dir_train = colab_path + partition_train + '/*/spchdatadir/*/*'
X_files_train = sorted(glob.glob(X_dir_train)) #Train files

X_dir_val = colab_path + partition_train + '/*/spchdatadir/*/*'
X_files_val = sorted(glob.glob(X_dir_train)) #Validation files

> **The purpose of the project is to convert audio recordings into phonemes and then classify the phonemes into intents.**

> *Each sequence of phonemes is mapped to one of 6 intents. The model should be able to read phoneme sequence and output an intent.*

In [26]:
"""
The dataset class reads sequnce of phonemes and a correspong intent.
The phonemes are mapped into indices using the above PHONEMES list
"""
class LibriSamples(torch.utils.data.Dataset):

    def __init__(self, recs, intents_lst, speakers_num, partition= "train"): 
        
        self.X_dir = colab_path + partition + '/*/spchdatadir/*/*'
        self.Y_dir = colab_path + partition + '/*/framedir/*/*'
        
        self.X_files = sorted(glob.glob(self.X_dir)) 
        self.Y_files = sorted(glob.glob(self.Y_dir))           
        
        X_files = records(self.X_files, 1)
        Y_files = records(self.Y_files, 1)
        
        x_files, y_files, self.intents = intents_func(intents_lst, X_files, Y_files)
        
        if partition == 'train':
            self.X_files, self.Y_files = choose_speakers(speakers, x_files, y_files, speakers_num)
        else:
            self.X_files, self.Y_files = x_files, y_files
                
        self.PHONEMES = PHONEMES
        assert(len(self.X_files) == len(self.Y_files))

    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, ind):
    
        X_path = self.X_files[ind] 
        Y_path = self.Y_files[ind] #Each file in Y_files has one of the intents in the list above

        X = np.load(X_path, allow_pickle=True) #Load numpy files of phonemes corresponding to each recording
        X_indices = [PHONEMES.index(xx) for xx in X] #Get index of each phoneme of loaded file
        
        f = open(Y_path) 
        r = f.read() #Read each intent
        Y_index = self.intents.index(r) #Index of each read intent

        f.close()
                
        return torch.LongTensor(X_indices), Y_index
    
    def collate_fn(self, batch):
        batch_x = [torch.tensor(x) for x,y in batch] 
        batch_y = [torch.tensor(y) for x,y in batch]
        batch_x_pad = pad_sequence(batch_x, batch_first=True, padding_value=0) #Utterances have variable length
        lengths_x = [len(x) for x,y in batch] #Store lenghths of all utterances

        return batch_x_pad, torch.tensor(batch_y), torch.tensor(lengths_x)

In [27]:
batch_size = 16

def data_prep(recs, intent_lst, speakers_num):
    train_data = LibriSamples(recs, intent_lst, speakers_num, 'train')
    val_data = LibriSamples(recs, intent_lst, speakers_num, 'validation')
    # test_data = LibriSamplesTest(root, 'test_order.csv')
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True, collate_fn=train_data.collate_fn)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=False, collate_fn=val_data.collate_fn)
    
    label_sizes = 0 # starting default

    if len(intent_lst) == 2:
        label_sizes = 2
        
    elif len(intent_lst) == 4:
        label_sizes = 4
        
    elif len(intent_lst) == 6:
        label_sizes = 6
    
    for data in val_loader:
        x, y, lx = data 
        break

    return train_loader, val_loader, x, y, lx, label_sizes

In [28]:
# Test code for checking shapes and return arguments of the train and val loaders
train_loader, val_loader, x, y, lx, label_sizes = data_prep(recs=7, intent_lst=intents_6, speakers_num=7)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [29]:
#Classifier architecture
class ICASSP2CNN(nn.Module):
    def __init__(self, vocab_size, label_size, embed_size=128, hidden_size=512, num_lstm_layers=2, bidirectional=False):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)

        self.batchnorm = nn.BatchNorm1d(2 * embed_size)

        self.lstm = nn.LSTM(input_size = 2 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional
                            )

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        input = self.embed(x)
        
        batch_size = input.size(0)
        input = input.transpose(1,2) 

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]
        
        logits = self.linear(h_n)

        return logits

In [30]:
model = ICASSP2CNN(len(PHONEMES), label_sizes).cuda() 
summary(model, x.to(device), lx)

              Kernel Shape   Output Shape   Params  Mult-Adds
Layer                                                        
0_embed         [128, 119]  [16, 25, 128]    15232      15232
1_cnn        [128, 128, 3]  [16, 128, 25]    49280    1228800
2_cnn2       [128, 128, 5]  [16, 128, 25]    82048    2048000
3_batchnorm          [256]  [16, 256, 25]      512        256
4_lstm                   -     [307, 512]  3678208    3670016
5_linear          [512, 6]        [16, 6]     3078       3072
-------------------------------------------------------------
                       Totals
Total params          3828358
Trainable params      3828358
Non-trainable params        0
Mult-Adds             6965376


/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embed,"[128, 119]","[16, 25, 128]",15232,15232
1_cnn,"[128, 128, 3]","[16, 128, 25]",49280,1228800
2_cnn2,"[128, 128, 5]","[16, 128, 25]",82048,2048000
3_batchnorm,[256],"[16, 256, 25]",512,256
4_lstm,-,"[307, 512]",3678208,3670016
5_linear,"[512, 6]","[16, 6]",3078,3072


In [31]:
criterion = torch.nn.CrossEntropyLoss() 

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

scaler = torch.cuda.amp.GradScaler()

In [32]:
def train_validate(train_loader, val_loader, label_size, batch_size=16):
    model = ICASSP2CNN(len(PHONEMES), label_sizes).cuda()

    criterion = torch.nn.CrossEntropyLoss() 

    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

    scaler = torch.cuda.amp.GradScaler()

    torch.cuda.empty_cache()
    epochs = 200
    max_valid_score = 0

    for epoch in range(epochs):
    
        model.train()

        batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

        num_correct = 0
        total_loss = 0
        
        for i, _data in enumerate(train_loader):
            x, y, input_lengths = _data
            data = x.float().to(device)
            y = y.long().to(device)
            
            optimizer.zero_grad()

            x = x.cuda()
            y = y.cuda()

            with torch.cuda.amp.autocast():     
                outputs = model(x, input_lengths)     
                loss = criterion(outputs, y)
            
            
            num_correct += int((torch.argmax(outputs,  axis=1) == y).sum())
            total_loss += float(loss)
            
            ls = torch.argmax(outputs, axis=1)
                
            batch_bar.set_postfix(
                acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
                loss="{:.04f}".format(float(total_loss / (i + 1))),
                num_correct=num_correct,
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            
            scaler.scale(loss).backward() 
            scaler.step(optimizer) 
            scaler.update() 

            batch_bar.update() 
        batch_bar.close() 

        print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
            epoch + 1,
            epochs,
            100 * num_correct / (len(train_loader) * batch_size),
            float(total_loss / len(train_loader)),
            float(optimizer.param_groups[0]['lr'])))
        
        model.eval()
        batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
        num_correct2 = 0
        for i, _data in enumerate(val_loader):
            x, y, input_lengths = _data
            x = x.cuda()
            y = y.cuda()

            with torch.no_grad():
                outputs = model(x, input_lengths)

            num_correct2 += int((torch.argmax(outputs,  axis=1) == y).sum())
            batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct2 / ((i + 1) * batch_size)))

            batch_bar.update()
        
        batch_bar.close()
        validation_score = 100 * num_correct2 / ((len(val_loader) * batch_size))
        print("Validation: {:.04f}%".format(validation_score))

                
        if validation_score > max_valid_score:
            max_valid_score = validation_score

    return max_valid_score

## Testing

In [33]:
# grid search
params = {
    'speakers': range(1, 8),
    'recordings': range(1, 8),
    'intents': zip(["Two"],[intents_2]),
}

In [34]:
intent_scores = {}
for name, intent in params['intents']:
    
    k_scores = {} # recordings
    for k in params['recordings']:
    
        s_scores = {} # speakers
        for s in params['speakers']:
            print(f'i: {name} k: {k} s: {s} phones: {len(PHONEMES)}')

            train_loader, val_loader, x, y, lx, label_size = data_prep(recs=k, intent_lst=intent, speakers_num=s)
            val_score = train_validate(train_loader, val_loader, label_size)

            s_scores.update({s: val_score})

        k_scores.update({k: s_scores})
    intent_scores.update({name: k_scores})

    # save before next
    import pandas as pd
    df = []
    df.append((name, pd.DataFrame(k_scores)))
    df[-1][1].to_csv(f"{df[-1][0]}.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


i: Two k: 1 s: 1 phones: 119


Epoch 1/200: Train Acc 0.0000%, Train Loss 1.8035, Learning Rate 0.0050


Validation: 0.0000%


Epoch 2/200: Train Acc 0.0000%, Train Loss 1.7993, Learning Rate 0.0050


...

Validation: 50.0000%
i: Two k: 2 s: 1 phones: 119


Epoch 1/200: Train Acc 0.0000%, Train Loss 1.8069, Learning Rate 0.0050


Validation: 0.0000%


Train:   0%|          | 0/1 [00:00<?, ?it/s, acc=0.0000%, loss=1.8022, lr=0.0050, num_correct=0]

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

df = []
for k, v in intent_scores.items():
    df.append((k, pd.DataFrame(v)))

df

In [ ]:
for item in df:
    item[1].to_csv(f"{item[0]}.csv")